In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

In [12]:

def get_month_dates(months):

    month_dates = []
    for month in months:
        files = [x for x in os.listdir("../data/bhavcopies/") if f"{month}" in x]
        dates = sorted([int(f[2:4]) for f in files])
        
        for d in dates:
            if d < 10:
                d = f"0{d}"
            month_dates.append(f"{d}-{month}")

    return month_dates

def get_weeks(dates):
    weeks = {}
    for i in range(1,14):
        weeks[i] = []
    
    for date in dates:
        day, month = date.split('-')
        date = datetime.datetime.strptime(f"{date}-2021", '%d-%b-%Y') 
        week_num = date.isocalendar()[1]
        weeks[week_num].append([date,day,month])
    
    return weeks

def get_equities(PATH, equities):
    curr_df = pd.read_csv(PATH)    
    curr_df = curr_df[curr_df['SYMBOL'].isin(equities)]
    curr_df = curr_df[curr_df['SERIES']=='EQ']

    return curr_df


In [51]:
MONTHS = ["JAN","FEB","MAR"]

NSE_500 = pd.read_csv("../data/bhavcopies/ind_nifty500list.csv")
NSE_500_SYMBOLS = list(NSE_500['Symbol'])
dates = get_month_dates(MONTHS)
weeks = get_weeks(dates[1:])

starting_df = get_equities(f"../data/bhavcopies/cm01{MONTHS[0]}2021bhav.csv", NSE_500_SYMBOLS)
starting_df['DATE'] = [datetime.datetime.strptime(x, '%d-%b-%Y') for x in starting_df['TIMESTAMP']]
starting_df['NAME'] = [x for x in starting_df['SYMBOL']]
starting_df['CONSECUTIVE'] = [0 for i in range(len(starting_df))]
starting_df['TYPE'] = ['None' for i in range(len(starting_df))]

starting_df = starting_df[['SYMBOL', 'NAME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'DATE', 'TYPE', 'CONSECUTIVE']]

highs = starting_df.set_index('SYMBOL').T.to_dict()
lows = starting_df.set_index('SYMBOL').T.to_dict()


for i in weeks:
        
    current_week = weeks[i]
    
    dataframes = []
    
    for single_day in current_week:
        dataframes.append(get_equities(f"../data/bhavcopies/cm{single_day[1]}{single_day[2]}2021bhav.csv", list(NSE_500_SYMBOLS)))
        
    new_df = pd.concat(dataframes)
    
    df_data = []
    for symbol in NSE_500_SYMBOLS:
        try:
            symbol_df = new_df[new_df['SYMBOL']==symbol]
            #symbol_df = symbol_df[['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE','TIMESTAMP']]

            
            weekly_open = symbol_df['OPEN'].values[0]
            weekly_close = symbol_df['CLOSE'].values[-1]
            weekly_high = np.max(symbol_df['HIGH'].values)
            weekly_low = np.min(symbol_df['LOW'].values)
            weekly_date = symbol_df['TIMESTAMP'].values[-1]

            weekly_data = [symbol, weekly_open, weekly_high, weekly_low, weekly_close, weekly_date]

            df_data.append(weekly_data)
        except:
            print(f"{symbol} not present in nifty 500 list")
        
    df = pd.DataFrame.from_records(df_data, columns=['SYMBOL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP'])
    print("Wrote for week " + str(weekly_date))
    df.to_csv(f"../data/weekly_nse_500/{weekly_date}.csv", index=False)
    
    

ATGL not present in nifty 500 list
SUZLON not present in nifty 500 list
Wrote for week 08-JAN-2021
SUZLON not present in nifty 500 list
Wrote for week 15-JAN-2021
SUZLON not present in nifty 500 list
Wrote for week 22-JAN-2021
SUZLON not present in nifty 500 list
Wrote for week 29-JAN-2021
Wrote for week 05-FEB-2021
Wrote for week 12-FEB-2021
Wrote for week 19-FEB-2021
Wrote for week 26-FEB-2021
Wrote for week 05-MAR-2021
Wrote for week 12-MAR-2021
Wrote for week 19-MAR-2021
Wrote for week 26-MAR-2021
Wrote for week 31-MAR-2021


In [20]:
MONTHS = ["JAN","FEB","MAR"]

NSE_500 = pd.read_csv("../data/bhavcopies/ind_nifty500list.csv")
NSE_500_SYMBOLS = list(NSE_500['Symbol'])
dates = get_month_dates(MONTHS)


starting_df = get_equities(f"../data/bhavcopies/cm01{MONTHS[0]}2021bhav.csv", NSE_500_SYMBOLS)
starting_df['DATE'] = [datetime.datetime.strptime(x, '%d-%b-%Y') for x in starting_df['TIMESTAMP']]
starting_df['NAME'] = [x for x in starting_df['SYMBOL']]
starting_df['CONSECUTIVE'] = [0 for i in range(len(starting_df))]
starting_df['TYPE'] = ['None' for i in range(len(starting_df))]

starting_df = starting_df[['SYMBOL', 'NAME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'DATE', 'TYPE', 'CONSECUTIVE']]

highs = starting_df.set_index('SYMBOL').T.to_dict()
lows = starting_df.set_index('SYMBOL').T.to_dict()

for i in range(2,len(dates)):
    third_day = dates[i]
    second_day = dates[i-1]
    first_day = dates[i-2]
    
    third_day_date, third_day_month = third_day.split('-')
    second_day_date, second_day_month = second_day.split('-')
    first_day_date, first_day_month = first_day.split('-')

    third_date = datetime.datetime.strptime(f"{third_day}-2021", '%d-%b-%Y')       
    second_date = datetime.datetime.strptime(f"{second_day}-2021", '%d-%b-%Y')
    first_date = datetime.datetime.strptime(f"{first_day}-2021", '%d-%b-%Y')

    
    
    third_df = get_equities(f"../data/bhavcopies/cm{third_day_date}{third_day_month}2021bhav.csv", list(NSE_500_SYMBOLS))
    second_df = get_equities(f"../data/bhavcopies/cm{second_day_date}{second_day_month}2021bhav.csv", list(NSE_500_SYMBOLS))
    first_df = get_equities(f"../data/bhavcopies/cm{first_day_date}{first_day_month}2021bhav.csv", list(NSE_500_SYMBOLS))

    
    for symbol in NSE_500_SYMBOLS:
        third_high = third_df[third_df['SYMBOL']==symbol].HIGH.values
        second_high = second_df[second_df['SYMBOL']==symbol].HIGH.values
        first_high = first_df[first_df['SYMBOL']==symbol].HIGH.values

        third_low = third_df[third_df['SYMBOL']==symbol].LOW.values
        second_low = second_df[second_df['SYMBOL']==symbol].LOW.values
        first_low = first_df[first_df['SYMBOL']==symbol].LOW.values

        if third_high < second_high and second_high < first_high and third_low > second_low and second_low > first_low:
            print(f"Narrow Range for {symbol} on {third_date}")
    
    
    
    

/home/rhino/anaconda3/envs/nba_api/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Narrow Range for CRISIL on 2021-01-05 00:00:00
Narrow Range for CENTURYPLY on 2021-01-05 00:00:00
Narrow Range for FINPIPE on 2021-01-05 00:00:00
Narrow Range for KANSAINER on 2021-01-05 00:00:00
Narrow Range for PGHH on 2021-01-05 00:00:00
Narrow Range for VINATIORGA on 2021-01-05 00:00:00
Narrow Range for BRIGADE on 2021-01-06 00:00:00
Narrow Range for CENTURYPLY on 2021-01-06 00:00:00
Narrow Range for IEX on 2021-01-06 00:00:00
Narrow Range for M&M on 2021-01-06 00:00:00
Narrow Range for TATAINVEST on 2021-01-06 00:00:00
Narrow Range for ALEMBICLTD on 2021-01-07 00:00:00
Narrow Range for BSOFT on 2021-01-07 00:00:00
Narrow Range for CARBORUNIV on 2021-01-07 00:00:00
Narrow Range for CERA on 2021-01-07 00:00:00
Narrow Range for GEPIL on 2021-01-07 00:00:00
Narrow Range for GESHIP on 2021-01-07 00:00:00
Narrow Range for MRPL on 2021-01-07 00:00:00
Narrow Range for PRSMJOHNSN on 2021-01-07 00:00:00
Narrow Range for SJVN on 2021-01-07 00:00:00
Narrow Range for SWANENERGY on 2021-01-07 0

Narrow Range for ABBOTINDIA on 2021-02-05 00:00:00
Narrow Range for HUHTAMAKI on 2021-02-05 00:00:00
Narrow Range for INDOCO on 2021-02-05 00:00:00
Narrow Range for INOXLEISUR on 2021-02-05 00:00:00
Narrow Range for LT on 2021-02-05 00:00:00
Narrow Range for OFSS on 2021-02-05 00:00:00
Narrow Range for DHANI on 2021-02-08 00:00:00
Narrow Range for JUBLFOOD on 2021-02-08 00:00:00
Narrow Range for KSCL on 2021-02-08 00:00:00
Narrow Range for MHRIL on 2021-02-08 00:00:00
Narrow Range for ORIENTCEM on 2021-02-08 00:00:00
Narrow Range for ORIENTELEC on 2021-02-08 00:00:00
Narrow Range for PIIND on 2021-02-08 00:00:00
Narrow Range for SEQUENT on 2021-02-08 00:00:00
Narrow Range for TRIDENT on 2021-02-08 00:00:00
Narrow Range for ASTRAL on 2021-02-09 00:00:00
Narrow Range for BALRAMCHIN on 2021-02-09 00:00:00
Narrow Range for CCL on 2021-02-09 00:00:00
Narrow Range for ENDURANCE on 2021-02-09 00:00:00
Narrow Range for GILLETTE on 2021-02-09 00:00:00
Narrow Range for IFBIND on 2021-02-09 00:00

Narrow Range for NAM-INDIA on 2021-03-09 00:00:00
Narrow Range for PIIND on 2021-03-09 00:00:00
Narrow Range for SFL on 2021-03-09 00:00:00
Narrow Range for ADVENZYMES on 2021-03-10 00:00:00
Narrow Range for ALOKINDS on 2021-03-10 00:00:00
Narrow Range for CUMMINSIND on 2021-03-10 00:00:00
Narrow Range for DCMSHRIRAM on 2021-03-10 00:00:00
Narrow Range for EIDPARRY on 2021-03-10 00:00:00
Narrow Range for ENDURANCE on 2021-03-10 00:00:00
Narrow Range for FDC on 2021-03-10 00:00:00
Narrow Range for GRSE on 2021-03-10 00:00:00
Narrow Range for IRB on 2021-03-10 00:00:00
Narrow Range for JSWENERGY on 2021-03-10 00:00:00
Narrow Range for MINDAIND on 2021-03-10 00:00:00
Narrow Range for SHREECEM on 2021-03-10 00:00:00
Narrow Range for TATACOMM on 2021-03-10 00:00:00
Narrow Range for ULTRACEMCO on 2021-03-10 00:00:00
Narrow Range for VTL on 2021-03-10 00:00:00
Narrow Range for BEML on 2021-03-12 00:00:00
Narrow Range for CENTURYTEX on 2021-03-12 00:00:00
Narrow Range for GODREJIND on 2021-03-